In [1]:
import tensorflow as tf

2025-08-29 21:30:19.108396: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-29 21:30:19.319266: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-29 21:30:20.856976: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
data = tf.keras.utils.image_dataset_from_directory(
    'test',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    )

Found 33 files belonging to 1 classes.


In [ ]:
from tensorflow.keras.datasets import te

In [5]:
import tensorflow as tf
import os

train_data = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=16,
    
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    )

Found 70295 files belonging to 38 classes.


In [4]:
val_data = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=16,
    
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    )

Found 17572 files belonging to 38 classes.


In [6]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
for x,y in train_data:
    print(x,x.shape)
    print(x,x.shape)
    break

tf.Tensor(
[[[[188.5  177.5  185.5 ]
   [189.5  178.5  186.5 ]
   [189.5  178.5  186.5 ]
   ...
   [182.75 176.75 180.75]
   [178.25 172.25 176.25]
   [180.5  174.5  178.5 ]]

  [[187.75 176.75 184.75]
   [189.   178.   186.  ]
   [189.   178.   186.  ]
   ...
   [177.5  171.5  175.5 ]
   [180.25 174.25 178.25]
   [180.5  174.5  178.5 ]]

  [[187.   176.   184.  ]
   [188.25 177.25 185.25]
   [188.25 177.25 185.25]
   ...
   [175.   169.   173.  ]
   [175.   169.   173.  ]
   [175.   169.   173.  ]]

  ...

  [[ 98.25  87.25  93.25]
   [ 94.    83.    89.  ]
   [ 95.75  84.75  90.75]
   ...
   [155.25 145.25 146.25]
   [153.5  143.5  144.5 ]
   [159.5  149.5  150.5 ]]

  [[ 98.75  87.75  93.75]
   [ 97.75  86.75  92.75]
   [ 97.    86.    92.  ]
   ...
   [162.   152.   153.  ]
   [158.75 148.75 149.75]
   [154.   144.   145.  ]]

  [[ 94.25  83.25  89.25]
   [ 97.25  86.25  92.25]
   [ 96.    85.    91.  ]
   ...
   [156.5  146.5  147.5 ]
   [153.5  143.5  144.5 ]
   [158.25 148.25 14

In [11]:
from tensorflow.keras.layers import Dense, conv2D
from tensorflow.keras.models import Sequentials

ImportError: cannot import name 'conv2D' from 'tensorflow.keras.layers' (/home/zer0/anaconda3/lib/python3.12/site-packages/keras/_tf_keras/keras/layers/__init__.py)